In [1]:
#
import pandas as pd
from datetime import datetime as dt
import numpy as np
import re
#
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
np.set_printoptions(suppress=True) # Suppress scientific notation where possible
#
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
# Load data and rename the columns
df = pd.read_csv("C:/Users/storm/Desktop/METIS/Project/project 3/forestfires.csv")


In [3]:
#commonly used data cleaning function if needed:

#Remove missing values
#df = df.replace('?', np.NaN).dropna().reset_index()

#change na values to 0
#df = df.fillna(0, inplace=True)

#Strip column name
#df.rename(columns=lambda x: x.strip(), inplace=True)

In [4]:
#Inspect dataset for anomaly
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [5]:
#Inspect dataset for anomaly
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [6]:
#Inspect dataset for anomaly
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [7]:
# Convert area column to binary denoting fire occur (1) or not occur (0)
df.area[df.area > 0] = 1
df.rename(columns={'area':'fire_breakout'}, inplace=True)


C:\Users\storm\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
#inspect results of binary conversion
df['fire_breakout'].value_counts()

1.0    270
0.0    247
Name: fire_breakout, dtype: int64

In [9]:
# Convert Day column to multi-binary 
df.loc[df['day'] == 'mon', 'day'] = 1
df.loc[df['day'] == 'tue', 'day'] = 2
df.loc[df['day'] == 'wed', 'day'] = 3
df.loc[df['day'] == 'thu', 'day'] = 4
df.loc[df['day'] == 'fri', 'day'] = 5
df.loc[df['day'] == 'sat', 'day'] = 6
df.loc[df['day'] == 'sun', 'day'] = 7

df.day = df.day.astype(int)
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,fire_breakout
0,7,5,mar,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [10]:
df['day'].value_counts()

7    95
5    85
6    84
1    74
2    64
4    61
3    54
Name: day, dtype: int64

In [11]:
#Different ways to convert month values to int

#method 1: create mapping dictionary
#d = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
#df.month = df.month.map(d)

In [12]:
# alternate method to convert month values used
df.month = pd.to_datetime(df.month, format='%b').dt.month
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,fire_breakout
0,7,5,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [13]:
df.shape

(517, 13)

In [ ]:
df.info()

In [15]:
#df.to_excel (r'Forestfires_cleaned.xlsx', index = False, header=True)
df.to_csv (r'Forestfires_cleaned.csv', index = False, header=True)